## Model - Neural Nets

### 1. Feature Engineering (continuing after data cleaning)

In [1]:
import pandas as pd
df = pd.read_pickle("../../data/processed/data_cleaned.pkl.gz",compression='gzip')

In [2]:
df.head()

,searchDate,flightDate,startingAirport,destinationAirport,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,segmentsDepartureTimeEpochSeconds_Leg1,...,segmentsDurationInSeconds_Leg4,segmentsDistance_Leg1,segmentsDistance_Leg2,segmentsDistance_Leg3,segmentsDistance_Leg4,segmentsCabinCode_Leg1,segmentsCabinCode_Leg2,segmentsCabinCode_Leg3,segmentsCabinCode_Leg4,traveltime_hours
0,2022-04-16,2022-04-17,ATL,BOS,False,False,True,248.600006,947.0,1.650215e+09,...,0.0,947.0,0.0,0.0,0.0,coach,0,0,0,2.483333
1,2022-04-16,2022-04-17,ATL,BOS,False,False,True,248.600006,947.0,1.650191e+09,...,0.0,947.0,0.0,0.0,0.0,coach,0,0,0,2.500000
2,2022-04-16,2022-04-17,ATL,BOS,False,False,True,248.600006,947.0,1.650210e+09,...,0.0,947.0,0.0,0.0,0.0,coach,0,0,0,2.500000
3,2022-04-16,2022-04-17,ATL,BOS,False,False,True,248.600006,947.0,1.650218e+09,...,0.0,947.0,0.0,0.0,0.0,coach,0,0,0,2.533333
4,2022-04-16,2022-04-17,ATL,BOS,False,False,True,248.600006,947.0,1.650204e+09,...,0.0,947.0,0.0,0.0,0.0,coach,0,0,0,2.566667


In [3]:
df['flightDate'].min(), df['flightDate'].max()

('2022-04-17', '2022-07-17')

In [4]:
df['segmentsCabinCode_Leg1'].value_counts()

coach            13472653
premium coach       22826
first               21120
business             3400
Name: segmentsCabinCode_Leg1, dtype: int64

In [5]:
# convert departure date to pandas datetime type
df['flightDate'] = pd.to_datetime(df['flightDate'])

### Cleaning time column

In [6]:
from dateutil import parser

# column containing parsed datetime strings as datetime.datetime objects
df['segmentsDepartureTimeRaw_Leg1'] = df['segmentsDepartureTimeRaw_Leg1'].apply(parser.parse)

# Extract 'H:M' format and store in a new column
df['departure_time'] = df['segmentsDepartureTimeRaw_Leg1'].apply(lambda x: x.strftime('%H:%M'))

In [7]:
# Rounding the 'H:M' formatted time to the nearest 30-minute interval
df['departure_time'] = pd.to_datetime(df['departure_time']).dt.round('30min').dt.strftime('%H:%M')

### Taking average price for all the flights for same dates, similar journey and timings

In [8]:
features = ['flightDate', 'startingAirport', 'destinationAirport', 'departure_time', 'segmentsCabinCode_Leg1']
target = 'totalFare'

# Grouping and calculating the average total fare for similar flights
grouped_data = df.groupby(features)[target].mean().reset_index()

In [9]:
grouped_data.dropna(subset=['totalFare'], inplace=True)

In [10]:
grouped_data.shape

(518989, 6)

In [11]:
grouped_data.head()

,flightDate,startingAirport,destinationAirport,departure_time,segmentsCabinCode_Leg1,totalFare
189,2022-04-17,ATL,BOS,05:30,coach,271.589996
193,2022-04-17,ATL,BOS,06:00,coach,252.600006
197,2022-04-17,ATL,BOS,06:30,coach,248.600006
201,2022-04-17,ATL,BOS,07:00,coach,251.100006
213,2022-04-17,ATL,BOS,08:30,coach,251.100006


In [13]:
grouped_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 518989 entries, 189 to 3956561
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   flightDate              518989 non-null  datetime64[ns]
 1   startingAirport         518989 non-null  category      
 2   destinationAirport      518989 non-null  category      
 3   departure_time          518989 non-null  object        
 4   segmentsCabinCode_Leg1  518989 non-null  category      
 5   totalFare               518989 non-null  float32       
dtypes: category(3), datetime64[ns](1), float32(1), object(1)
memory usage: 15.3+ MB


In [14]:
grouped_data['flightDate'].min(), grouped_data['flightDate'].max()

(Timestamp('2022-04-17 00:00:00'), Timestamp('2022-07-17 00:00:00'))

In [15]:
# rename column to cabin_type
grouped_data.rename(columns = {'segmentsCabinCode_Leg1':'cabin_type'}, inplace = True)

In [16]:
# Save the processed data file to csv
grouped_data.to_csv("../../data/processed/data_processed.csv", index=False)

### Extracting date and time features
#### Reading saved data

In [1]:
import pandas as pd
flight_data = pd.read_csv("../../data/processed/data_processed.csv")

In [2]:
## convert into pandas data type
dtype_mapping = {'startingAirport': 'category',
        'destinationAirport': 'category',
        'totalFare': 'float32',
        'cabin_type': 'category'}
flight_data = flight_data.astype(dtype_mapping)

In [3]:
import sys
sys.path.append("../../src/features")

# import function from build_features python script 
from build_features import get_date_features, get_time_features
flight_data = get_date_features(flight_data)
flight_data = get_time_features(flight_data)

/Users/kritika/Desktop/MDSI/AMLA/adv_mla_at3/notebooks/Kritika/../../src/features/build_features.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  flight_data['departure_time'] = pd.to_datetime(flight_data['departure_time'])


In [4]:
flight_data.head()

,startingAirport,destinationAirport,cabin_type,totalFare,month,day,weekday,departure_time_sin,departure_time_cos,departure_time_category
0,ATL,BOS,coach,271.589996,4,17,6,0.965926,2.588190e-01,night
1,ATL,BOS,coach,252.600006,4,17,6,1.000000,6.123234e-17,night
2,ATL,BOS,coach,248.600006,4,17,6,1.000000,6.123234e-17,night
3,ATL,BOS,coach,251.100006,4,17,6,0.965926,-2.588190e-01,morning
4,ATL,BOS,coach,251.100006,4,17,6,0.866025,-5.000000e-01,morning


In [5]:
flight_data.describe()

,totalFare,month,day,weekday,departure_time_sin,departure_time_cos
count,518989.000000,518989.000000,518989.000000,518989.000000,518989.000000,5.189890e+05
mean,409.114044,5.518647,15.690963,3.000399,-0.046502,-3.360548e-01
std,268.047119,0.921477,8.733479,2.010741,0.739494,5.814252e-01
min,23.969999,4.000000,1.000000,0.000000,-1.000000,-1.000000e+00
25%,249.028580,5.000000,8.000000,1.000000,-0.707107,-8.660254e-01
50%,359.314270,5.000000,16.000000,3.000000,-0.258819,-5.000000e-01
75%,499.096008,6.000000,23.000000,5.000000,0.707107,6.123234e-17
max,8260.610352,7.000000,31.000000,6.000000,1.000000,1.000000e+00


### 2. Modelling

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Splitting into features and target
X = flight_data.drop('totalFare', axis=1)
y = flight_data['totalFare']

# Label encoding categorical variables
label_encoders = {}
for col in ['startingAirport', 'destinationAirport', 'cabin_type', 'departure_time_category']:
    label_encoders[col] = LabelEncoder()
    X[col] = label_encoders[col].fit_transform(X[col])

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=23)

# Scale the data using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
### Save label encoder and standard scaler models
import joblib

joblib.dump(label_encoders, '../../models/Kritika/label_encoder.joblib')
joblib.dump(scaler, '../../models/Kritika/standard_scaler.joblib')

['../../models/Kritika/standard_scaler.joblib']

In [7]:
X_train.shape, X_test.shape

((389241, 9), (129748, 9))

### 2.1 Accessing Baseline

In [8]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Calculate the mean of totalFare
mean_total_fare = y_train.mean()

# Create a list with the mean value to match the length of the test set
baseline_predictions = [mean_total_fare] * len(X_test)

# Calculate the Mean Absolute Error of the baseline model
rmse_baseline = np.sqrt(mean_squared_error(y_test, baseline_predictions))
print(f"Root Mean Squared Error of the baseline model: {rmse_baseline}")

Root Mean Squared Error of the baseline model: 268.4026184082031


**Reasons to choose specific parameters a/c to business objective:**

1. Loss Function:
A loss function that penalizes underestimation more than overestimation - The Mean Squared Logarithmic Error (MSLE) is one such loss function that penalizes underestimation of higher values more than lower values. It's particularly useful when dealing with data with exponential growth patterns or when a linear model is desired on the logarithm of the predicted and actual values.

2. Optimizer:
An optimizer that balances speed and accuracy, such as Adam (Adaptive Moment Estimation), is commonly a good choice. Adam adapts learning rates for each parameter and combines the advantages of both AdaGrad and RMSProp.

3. Metrics:
For evaluation, while RMSE (Root Mean Squared Error) is a standard metric for regression tasks, i my case, it's crucial to also monitor the Mean Absolute Percentage Error (MAPE) or the Percentage of Absolute Errors (PAE). These metrics show the percentage of the absolute difference between the predicted and actual values and could help ensure that the model doesn't make significant underestimations.

### 2.2 Neural Networks model
### Experiment 2 - Added Dropout and model checkpoint

In [6]:
### Call training helper script
import sys
sys.path.append('../../src/models')

**Load train_model.py script to call train_and_evaluate_model function** 

In [9]:
from train_model import train_and_evaluate_model
train_and_evaluate_model(X_train_scaled, y_train, X_test_scaled, y_test, "exp2_best_model.h5")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kritz2399 (kritika_23). Use `wandb login --relogin` to force relogin


Epoch 1/100
12164/12164 [==============================] - 9s 686us/step - loss: 0.4193 - mean_squared_error: 64682.7773 - mean_absolute_percentage_error: 45.6099 - val_loss: 0.2244 - val_mean_squared_error: 54868.7500 - val_mean_absolute_percentage_error: 41.7694 - lr: 0.0010
Epoch 2/100
  252/12164 [..............................] - ETA: 7s - loss: 0.2493 - mean_squared_error: 58094.6953 - mean_absolute_percentage_error: 44.4085

/Users/kritika/Library/Caches/pypoetry/virtualenvs/adv-mla-at3-NzhSlUn1-py3.9/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12164/12164 [==============================] - 8s 686us/step - loss: 0.2425 - mean_squared_error: 54027.6836 - mean_absolute_percentage_error: 43.8545 - val_loss: 0.2154 - val_mean_squared_error: 46061.1641 - val_mean_absolute_percentage_error: 42.3332 - lr: 0.0010
Epoch 3/100
12164/12164 [==============================] - 9s 701us/step - loss: 0.2361 - mean_squared_error: 50771.5000 - mean_absolute_percentage_error: 43.1355 - val_loss: 0.2098 - val_mean_squared_error: 43971.7852 - val_mean_absolute_percentage_error: 41.5720 - lr: 0.0010
Epoch 4/100
12164/12164 [==============================] - 8s 692us/step - loss: 0.2275 - mean_squared_error: 48722.9766 - mean_absolute_percentage_error: 41.8491 - val_loss: 0.1990 - val_mean_squared_error: 42185.6289 - val_mean_absolute_percentage_error: 39.4412 - lr: 0.0010
Epoch 5/100
12164/12164 [==============================] - 8s 697us/step - loss: 0.2210 - mean_squared_error: 47062.2852 - mean_absolute_percentage_error: 40.9139 - val_loss: 0.1

12164/12164 [==============================] - 8s 689us/step - loss: 0.1263 - mean_squared_error: 29678.8848 - mean_absolute_percentage_error: 28.8735 - val_loss: 0.1107 - val_mean_squared_error: 26090.5996 - val_mean_absolute_percentage_error: 26.9943 - lr: 0.0010
Epoch 32/100
12164/12164 [==============================] - 8s 690us/step - loss: 0.1249 - mean_squared_error: 29339.6484 - mean_absolute_percentage_error: 28.6515 - val_loss: 0.1074 - val_mean_squared_error: 25675.0996 - val_mean_absolute_percentage_error: 26.1990 - lr: 0.0010
Epoch 33/100
12164/12164 [==============================] - 8s 688us/step - loss: 0.1243 - mean_squared_error: 29187.0918 - mean_absolute_percentage_error: 28.5716 - val_loss: 0.1099 - val_mean_squared_error: 25078.9023 - val_mean_absolute_percentage_error: 27.7171 - lr: 0.0010
Epoch 34/100
12164/12164 [==============================] - 8s 690us/step - loss: 0.1232 - mean_squared_error: 29021.7207 - mean_absolute_percentage_error: 28.3766 - val_loss: 

12164/12164 [==============================] - 8s 679us/step - loss: 0.1115 - mean_squared_error: 27266.9980 - mean_absolute_percentage_error: 26.5891 - val_loss: 0.0983 - val_mean_squared_error: 23895.2578 - val_mean_absolute_percentage_error: 24.7867 - lr: 0.0010
Epoch 61/100
12164/12164 [==============================] - 8s 685us/step - loss: 0.1113 - mean_squared_error: 27178.3066 - mean_absolute_percentage_error: 26.5470 - val_loss: 0.0969 - val_mean_squared_error: 24207.0957 - val_mean_absolute_percentage_error: 24.3761 - lr: 0.0010
Epoch 62/100
12164/12164 [==============================] - 8s 682us/step - loss: 0.1109 - mean_squared_error: 27079.5957 - mean_absolute_percentage_error: 26.5063 - val_loss: 0.0979 - val_mean_squared_error: 23232.4570 - val_mean_absolute_percentage_error: 25.0702 - lr: 0.0010
Epoch 63/100
12164/12164 [==============================] - 8s 687us/step - loss: 0.1059 - mean_squared_error: 26031.8574 - mean_absolute_percentage_error: 25.7327 - val_loss: 

12164/12164 [==============================] - 8s 667us/step - loss: 0.1022 - mean_squared_error: 25213.4629 - mean_absolute_percentage_error: 25.1739 - val_loss: 0.0899 - val_mean_squared_error: 22158.0625 - val_mean_absolute_percentage_error: 23.5454 - lr: 1.0000e-04
Epoch 90/100
12164/12164 [==============================] - 8s 671us/step - loss: 0.1022 - mean_squared_error: 25184.7402 - mean_absolute_percentage_error: 25.1638 - val_loss: 0.0895 - val_mean_squared_error: 21978.8945 - val_mean_absolute_percentage_error: 23.3517 - lr: 1.0000e-04
Epoch 91/100
12164/12164 [==============================] - 8s 673us/step - loss: 0.1023 - mean_squared_error: 25257.5293 - mean_absolute_percentage_error: 25.1873 - val_loss: 0.0899 - val_mean_squared_error: 21965.3184 - val_mean_absolute_percentage_error: 23.5720 - lr: 1.0000e-04
Epoch 92/100
12164/12164 [==============================] - 8s 674us/step - loss: 0.1022 - mean_squared_error: 25241.3164 - mean_absolute_percentage_error: 25.1813 

4055/4055 [==============================] - 1s 331us/step


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


test_rmse,▁
train_rmse,▁
val_mse,█▆▅▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_rmse,147.72517
train_rmse,146.22525
val_mse,21864.125


### Experiment 3 - Added extra hidden layer

In [8]:
from train_model import train_and_evaluate_model
train_and_evaluate_model(X_train_scaled, y_train, X_test_scaled, y_test, "exp3_best_model.h5")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kritz2399 (kritika_23). Use `wandb login --relogin` to force relogin


Epoch 1/100
12164/12164 [==============================] - 9s 729us/step - loss: 0.4020 - mean_squared_error: 65696.6094 - mean_absolute_percentage_error: 47.2935 - val_loss: 0.2235 - val_mean_squared_error: 53178.5312 - val_mean_absolute_percentage_error: 43.2018 - lr: 0.0010
Epoch 2/100
  244/12164 [..............................] - ETA: 7s - loss: 0.2648 - mean_squared_error: 57635.6328 - mean_absolute_percentage_error: 45.9418

/Users/kritika/Library/Caches/pypoetry/virtualenvs/adv-mla-at3-NzhSlUn1-py3.9/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12164/12164 [==============================] - 9s 751us/step - loss: 0.2571 - mean_squared_error: 56657.7812 - mean_absolute_percentage_error: 44.9860 - val_loss: 0.2100 - val_mean_squared_error: 48079.7461 - val_mean_absolute_percentage_error: 40.0231 - lr: 0.0010
Epoch 3/100
12164/12164 [==============================] - 9s 768us/step - loss: 0.2441 - mean_squared_error: 49255.6836 - mean_absolute_percentage_error: 43.4431 - val_loss: 0.2023 - val_mean_squared_error: 41084.0156 - val_mean_absolute_percentage_error: 39.8869 - lr: 0.0010
Epoch 4/100
12164/12164 [==============================] - 10s 784us/step - loss: 0.2331 - mean_squared_error: 46263.8672 - mean_absolute_percentage_error: 42.1615 - val_loss: 0.1969 - val_mean_squared_error: 41577.9492 - val_mean_absolute_percentage_error: 38.5500 - lr: 0.0010
Epoch 5/100
12164/12164 [==============================] - 9s 731us/step - loss: 0.2253 - mean_squared_error: 45303.5547 - mean_absolute_percentage_error: 41.2494 - val_loss: 0.

12164/12164 [==============================] - 9s 727us/step - loss: 0.1236 - mean_squared_error: 30013.8691 - mean_absolute_percentage_error: 28.1943 - val_loss: 0.1054 - val_mean_squared_error: 26335.6758 - val_mean_absolute_percentage_error: 25.4250 - lr: 0.0010
Epoch 32/100
12164/12164 [==============================] - 9s 721us/step - loss: 0.1230 - mean_squared_error: 29947.0332 - mean_absolute_percentage_error: 28.0983 - val_loss: 0.1026 - val_mean_squared_error: 25157.5332 - val_mean_absolute_percentage_error: 25.3693 - lr: 0.0010
Epoch 33/100
12164/12164 [==============================] - 9s 722us/step - loss: 0.1171 - mean_squared_error: 28669.3633 - mean_absolute_percentage_error: 27.2561 - val_loss: 0.0983 - val_mean_squared_error: 24924.0762 - val_mean_absolute_percentage_error: 24.4696 - lr: 2.0000e-04
Epoch 34/100
12164/12164 [==============================] - 9s 720us/step - loss: 0.1164 - mean_squared_error: 28499.3516 - mean_absolute_percentage_error: 27.1598 - val_lo

4055/4055 [==============================] - 1s 334us/step


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


test_rmse,▁
train_rmse,▁
val_mse,█▇▅▅▅▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_rmse,154.16371
train_rmse,152.09752
val_mse,23733.875


### Experiment 4 - Simply the model as complexity is increasing the error

In [8]:
from train_model import train_and_evaluate_model
train_and_evaluate_model(X_train_scaled, y_train, X_test_scaled, y_test, "exp4_best_model.h5")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kritz2399 (kritika_23). Use `wandb login --relogin` to force relogin


Epoch 1/100
12164/12164 [==============================] - 8s 617us/step - loss: 0.6828 - mean_squared_error: 75166.7656 - mean_absolute_percentage_error: 47.3909 - val_loss: 0.2279 - val_mean_squared_error: 56841.2891 - val_mean_absolute_percentage_error: 43.2782 - lr: 0.0010
Epoch 2/100
  275/12164 [..............................] - ETA: 6s - loss: 0.2519 - mean_squared_error: 60116.8984 - mean_absolute_percentage_error: 44.8965

/Users/kritika/Library/Caches/pypoetry/virtualenvs/adv-mla-at3-NzhSlUn1-py3.9/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12164/12164 [==============================] - 7s 598us/step - loss: 0.2491 - mean_squared_error: 59358.5039 - mean_absolute_percentage_error: 44.5227 - val_loss: 0.2247 - val_mean_squared_error: 55263.4766 - val_mean_absolute_percentage_error: 42.2894 - lr: 0.0010
Epoch 3/100
12164/12164 [==============================] - 7s 602us/step - loss: 0.2448 - mean_squared_error: 57000.7266 - mean_absolute_percentage_error: 44.0759 - val_loss: 0.2206 - val_mean_squared_error: 51605.8320 - val_mean_absolute_percentage_error: 41.9433 - lr: 0.0010
Epoch 4/100
12164/12164 [==============================] - 7s 598us/step - loss: 0.2417 - mean_squared_error: 54233.4883 - mean_absolute_percentage_error: 43.8316 - val_loss: 0.2182 - val_mean_squared_error: 49327.3320 - val_mean_absolute_percentage_error: 41.7136 - lr: 0.0010
Epoch 5/100
12164/12164 [==============================] - 7s 598us/step - loss: 0.2401 - mean_squared_error: 53246.7734 - mean_absolute_percentage_error: 43.7044 - val_loss: 0.2

12164/12164 [==============================] - 7s 594us/step - loss: 0.2325 - mean_squared_error: 50970.1836 - mean_absolute_percentage_error: 42.9412 - val_loss: 0.2155 - val_mean_squared_error: 46597.5664 - val_mean_absolute_percentage_error: 42.0112 - lr: 1.0000e-04
Epoch 32/100
12164/12164 [==============================] - 7s 593us/step - loss: 0.2325 - mean_squared_error: 50812.8398 - mean_absolute_percentage_error: 42.9379 - val_loss: 0.2155 - val_mean_squared_error: 47016.1523 - val_mean_absolute_percentage_error: 41.6959 - lr: 1.0000e-04
Epoch 33/100
12164/12164 [==============================] - 7s 598us/step - loss: 0.2323 - mean_squared_error: 50901.6094 - mean_absolute_percentage_error: 42.8887 - val_loss: 0.2154 - val_mean_squared_error: 46810.3398 - val_mean_absolute_percentage_error: 41.8476 - lr: 1.0000e-04
Epoch 34/100
12164/12164 [==============================] - 7s 600us/step - loss: 0.2319 - mean_squared_error: 50686.4961 - mean_absolute_percentage_error: 42.8921 

4055/4055 [==============================] - 1s 292us/step


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


test_rmse,▁
train_rmse,▁
val_mse,█▇▄▃▂▂▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_rmse,216.34686
train_rmse,216.26765
val_mse,46815.25


**Note: Amongst the 4 experiments performed, experiment 2 has the best train and test results. The best model has been saved as `exp2_best_model.h5` in the /models folder**

### Appendix - Experiment 1 

In [9]:
import wandb
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Initialize wandb
wandb.init(project="adv_mla_at3")

# Log hyperparameters
config = wandb.config
config.epochs = 50
config.batch_size = 32
config.optimizer = 'adam'
config.loss = 'mean_squared_error'
config.metrics = ['root_mean_squared_error']

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kritz2399 (kritika_23). Use `wandb login --relogin` to force relogin


In [11]:
# Define and compile your model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(X_train.columns)]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError()])

# Train the model
results = model.fit(X_train_scaled, y_train, epochs=config.epochs, 
                    batch_size=config.batch_size, 
                    validation_data=(X_test_scaled, y_test),
                   verbose=True)

# Make predictions on train and test sets
train_predictions = model.predict(X_train_scaled)
test_predictions = model.predict(X_test_scaled)

# Calculate RMSE for train and test sets
train_rmse = mean_squared_error(y_train, train_predictions, squared=False)
test_rmse = mean_squared_error(y_test, test_predictions, squared=False)

# Log RMSE scores
wandb.log({"train_rmse": train_rmse, "test_rmse": test_rmse})

# Log model weights
wandb.log({"model_weights": model.get_weights()})

wandb.finish()

Epoch 1/50
12164/12164 [==============================] - 8s 602us/step - loss: 0.6129 - mean_squared_error: 71112.0938 - mean_absolute_percentage_error: 44.9520 - val_loss: 0.2273 - val_mean_squared_error: 57234.6875 - val_mean_absolute_percentage_error: 42.4495 - lr: 0.0010
Epoch 2/50
12164/12164 [==============================] - 7s 589us/step - loss: 0.2247 - mean_squared_error: 55934.9844 - mean_absolute_percentage_error: 42.1586 - val_loss: 0.2237 - val_mean_squared_error: 55383.9570 - val_mean_absolute_percentage_error: 41.9155 - lr: 0.0010
Epoch 3/50
12164/12164 [==============================] - 7s 588us/step - loss: 0.2213 - mean_squared_error: 54261.8945 - mean_absolute_percentage_error: 41.7429 - val_loss: 0.2205 - val_mean_squared_error: 53080.2422 - val_mean_absolute_percentage_error: 41.9519 - lr: 0.0010
Epoch 4/50
12164/12164 [==============================] - 7s 604us/step - loss: 0.2180 - mean_squared_error: 52084.3555 - mean_absolute_percentage_error: 41.3575 - val_l

12164/12164 [==============================] - 7s 609us/step - loss: 0.1880 - mean_squared_error: 39660.1133 - mean_absolute_percentage_error: 37.0817 - val_loss: 0.1892 - val_mean_squared_error: 39634.1562 - val_mean_absolute_percentage_error: 37.5732 - lr: 2.0000e-04
Epoch 31/50
12164/12164 [==============================] - 7s 599us/step - loss: 0.1879 - mean_squared_error: 39659.2617 - mean_absolute_percentage_error: 37.0621 - val_loss: 0.1890 - val_mean_squared_error: 39733.3750 - val_mean_absolute_percentage_error: 37.5556 - lr: 2.0000e-04
Epoch 32/50
12164/12164 [==============================] - 7s 587us/step - loss: 0.1878 - mean_squared_error: 39650.7695 - mean_absolute_percentage_error: 37.0587 - val_loss: 0.1891 - val_mean_squared_error: 39916.4883 - val_mean_absolute_percentage_error: 37.1469 - lr: 2.0000e-04
Epoch 33/50
12164/12164 [==============================] - 7s 610us/step - loss: 0.1878 - mean_squared_error: 39646.7461 - mean_absolute_percentage_error: 37.0491 - v

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


test_rmse,▁
train_rmse,▁
test_rmse,200.37106
train_rmse,199.76857
